In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from harness import RMSE

In [2]:
from sklearn.cross_validation import train_test_split
import xgboost as xgb

In [14]:
##Loading training set
train_df = pd.read_csv('../data/train_new_features.csv')

In [15]:
def elem_counts(df):
    df['len_smiles'] = df['smiles'].str.len()
    elements = ['nH', 'n', 'c', 'c1', 'Si', 'SiH2', '=', '-', 'CC', 'ncc', 'C1', 'C', 'H', 'cc', 'ccc', 'cccc', 'cc1',\
           '(C1)', '(c1)', '(o1)', '(s1)', 'nc', 'c12', 'c2', 'c1cc', '(cc1)', 'c2C', 'cc3', 'oc', 'ncc', 'C1=C',\
                'C=c', 'C=C', 'ccn', 'c3', '[se]', '=CCC=', 'c21', 'c1c', 'cn', 'c4c', 'c3c', 'coc',\
               'ccccc', '[SiH2]C', 'cc4', 'sc', 'cccnc', 'cnc', 'scc', 'c1s', 'cc4', 'sc2', '2c2', 'c5', 'c6',\
               'c2c', '[nH]c', 'cnc4', 'C1=C', 'Cc', 'nsnc', 'sccc', 'cocc', '(o2)', '(cn1)']
    iterator = 0
    for elem in elements:
        col_name = 'count_' + str(iterator)
        df[col_name] = df['smiles'].str.count(elem)
        iterator += 1
    return df

In [16]:
train_df = elem_counts(train_df)

In [17]:
train_cols = train_df.columns
train_cols = train_cols.difference(['smiles', 'gap', 'Unnamed: 0'])
X = train_df[train_cols]
y = train_df['gap']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1)
print X_train.shape, X_test.shape

(900000, 108) (100000, 108)


In [25]:
####Split into validation and training list
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=1)

In [26]:
dtrain = xgb.DMatrix(X_train, label=y_train)

In [27]:
dval = xgb.DMatrix(X_val, label=y_val)

In [84]:
param = {'bst:max_depth':10, 'bst:eta':1, 'bst:min_child_weight':0.9, 'silent':1, 'objective':'reg:linear', 'nthreads':4 }
param['eval_metric'] = 'rmse'

In [85]:
evallist  = [(dval,'eval'), (dtrain,'train')]

In [86]:
num_round = 10
bst = xgb.train( param, dtrain, num_round, evallist )

[0]	eval-rmse:0.185110	train-rmse:0.183348
[1]	eval-rmse:0.171535	train-rmse:0.168855
[2]	eval-rmse:0.160881	train-rmse:0.157420
[3]	eval-rmse:0.156310	train-rmse:0.152266
[4]	eval-rmse:0.150974	train-rmse:0.146312
[5]	eval-rmse:0.148647	train-rmse:0.143514
[6]	eval-rmse:0.145893	train-rmse:0.140089
[7]	eval-rmse:0.144890	train-rmse:0.138681
[8]	eval-rmse:0.142372	train-rmse:0.135573
[9]	eval-rmse:0.141279	train-rmse:0.134160


In [92]:
tree = xgb.XGBRegressor()

In [98]:
tree.fit(X_train,y_train, eval_metric="rmse",early_stopping_rounds=10,eval_set=[(X_val, y_val)])

In [96]:
#dtest = xgb.DMatrix(X_test)
ypred = tree.predict(X_test)

In [97]:
RMSE(y_test, ypred)

RMSE =  0.185053335332


0.18505333533203822

In [75]:
## Readying test data
test_df = pd.read_csv('../data/test_new_features.csv')
test_df = elem_counts(test_df)
test_cols = test_df.columns
test_cols = test_cols.difference(['smiles', 'smiles_mol', 'Unnamed: 0'])
test_data = test_df[train_cols]

In [76]:
test_pred = bst.predict(xgb.DMatrix(test_data))

In [78]:
#Saving to file
out_df = pd.DataFrame({'Id':np.array(test_df.index), 'Prediction': test_pred})
out_df['Id'] = out_df['Id'] + 1
out_df = out_df.set_index('Id')
out_df.Prediction = out_df.Prediction.astype(float)

In [79]:
out_df.to_csv('../data/abhi_trial_3.csv')